In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stanford-covid-vaccine/test.json
/kaggle/input/stanford-covid-vaccine/sample_submission.csv
/kaggle/input/stanford-covid-vaccine/train.json
/kaggle/input/stanford-covid-vaccine/bpps/id_39d8f1120.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_fc0bf0cd5.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_fcf051e33.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_459a0aa1c.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_8a7896b66.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_9ca4c4379.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_e5f1fbce7.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_136f22289.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_e3c2dfe4d.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_d57b2913d.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_44e9f5490.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_d519cf4ab.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_946856a77.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_e8550ca77.npy
/kaggle/input/stanfo

In [2]:
#!pip install seaborn
# !pip install --upgrade catboost
import catboost
import optuna
import imblearn
from catboost import CatBoostRegressor
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
import pandas as pd
from catboost import *
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import Pool
from datetime import datetime
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LinearRegression,RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from scipy.stats import norm,skew
from scipy import stats
from sklearn.metrics import mean_squared_error,make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
import pandas as pd
import nltk
import operator
import re
import sys
from scipy import stats
from nltk.corpus import stopwords
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
# from multiprocessing import Pool
nltk.download("stopwords")
nltk.download("punkt")
import statsmodels.api as sm
from statsmodels.formula.api import ols
import time

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
train = pd.read_json('../input/stanford-covid-vaccine/train.json',lines=True)
test = pd.read_json('../input/stanford-covid-vaccine/test.json', lines=True)
ss = pd.read_csv('../input/stanford-covid-vaccine/sample_submission.csv')
train.shape, test.shape, ss.shape

((2400, 19), (3634, 7), (457953, 6))

In [4]:
train.columns

Index(['index', 'id', 'sequence', 'structure', 'predicted_loop_type',
       'signal_to_noise', 'SN_filter', 'seq_length', 'seq_scored',
       'reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10',
       'deg_error_Mg_50C', 'deg_error_50C', 'reactivity', 'deg_Mg_pH10',
       'deg_pH10', 'deg_Mg_50C', 'deg_50C'],
      dtype='object')

In [5]:
test.head()

,index,id,sequence,structure,predicted_loop_type,seq_length,seq_scored
0,0,id_00073f8be,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68
1,1,id_000ae4237,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91
2,2,id_00131c573,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,...........((.(((.(.(..((..((..((((...))))..))...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,107,68
3,3,id_00181fd34,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,......((((((((((....))))))))))((((((..((.(((.....,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,107,68
4,4,id_0020473f7,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91


In [6]:
# S: paired "Stem" M: Multiloop I: Internal loop B: Bulge H: Hairpin loop E: dangling End X: eXternal loop

In [7]:
ss.columns

Index(['id_seqpos', 'reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C',
       'deg_50C'],
      dtype='object')

In [8]:
test['E']=[sum([i=='E' for i in j])/len(j) for j in test['predicted_loop_type']]
test['S']=[sum([i=='S' for i in j])/len(j) for j in test['predicted_loop_type']]
test['B']=[sum([i=='B' for i in j])/len(j) for j in test['predicted_loop_type']]
test['H']=[sum([i=='H' for i in j])/len(j) for j in test['predicted_loop_type']]
test['I']=[sum([i=='I' for i in j])/len(j) for j in test['predicted_loop_type']]
test['X']=[sum([i=='X' for i in j])/len(j) for j in test['predicted_loop_type']]
test['M']=[sum([i=='M' for i in j])/len(j) for j in test['predicted_loop_type']]

test['G']=[sum([i=='G' for i in j])/len(j) for j in test['sequence']]
test['A']=[sum([i=='A' for i in j])/len(j) for j in test['sequence']]
test['C']=[sum([i=='C' for i in j])/len(j) for j in test['sequence']]
test['U']=[sum([i=='U' for i in j])/len(j) for j in test['sequence']]
test['Paired']=[sum([i=='(' or i==')' for i in j])/len(j) for j in test['structure']]
test['Unpaired']=[sum([i=='.' for i in j])/len(j) for j in test['structure']]


In [9]:
train['E']=[sum([i=='E' for i in j])/len(j) for j in train['predicted_loop_type']]
train['S']=[sum([i=='S' for i in j])/len(j) for j in train['predicted_loop_type']]
train['B']=[sum([i=='B' for i in j])/len(j) for j in train['predicted_loop_type']]
train['H']=[sum([i=='H' for i in j])/len(j) for j in train['predicted_loop_type']]
train['I']=[sum([i=='I' for i in j])/len(j) for j in train['predicted_loop_type']]
train['X']=[sum([i=='X' for i in j])/len(j) for j in train['predicted_loop_type']]
train['M']=[sum([i=='M' for i in j])/len(j) for j in train['predicted_loop_type']]
train['G']=[sum([i=='G' for i in j])/len(j) for j in train['sequence']]
train['A']=[sum([i=='A' for i in j])/len(j) for j in train['sequence']]
train['C']=[sum([i=='C' for i in j])/len(j) for j in train['sequence']]
train['U']=[sum([i=='U' for i in j])/len(j) for j in train['sequence']]
train['Paired']=[sum([i=='(' or i==')' for i in j])/len(j) for j in train['structure']]
train['Unpaired']=[sum([i=='.' for i in j])/len(j) for j in train['structure']]

In [10]:
train.columns

Index(['index', 'id', 'sequence', 'structure', 'predicted_loop_type',
       'signal_to_noise', 'SN_filter', 'seq_length', 'seq_scored',
       'reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10',
       'deg_error_Mg_50C', 'deg_error_50C', 'reactivity', 'deg_Mg_pH10',
       'deg_pH10', 'deg_Mg_50C', 'deg_50C', 'E', 'S', 'B', 'H', 'I', 'X', 'M',
       'G', 'A', 'C', 'U', 'Paired', 'Unpaired'],
      dtype='object')

In [11]:
# train['reactivity']=[np.mean(x) for x in train['reactivity']]
# train['deg_error_Mg_pH10']=[np.mean(x) for x in train['deg_Mg_pH10']]
# train['deg_error_pH10']=[np.mean(x) for x in train['deg_pH10']]
# train['deg_error_Mg_50C']=[np.mean(x) for x in train['deg_Mg_50C']]
# train['deg_error_50C']=[np.mean(x) for x in train['deg_50C']]


In [12]:
# train=train[['id', 'sequence', 'structure', 'predicted_loop_type',
#        'signal_to_noise', 'SN_filter', 'seq_length', 'seq_scored',
#        'reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10',
#        'deg_error_Mg_50C', 'deg_error_50C', 'reactivity', 'deg_Mg_pH10',
#        'deg_pH10', 'deg_Mg_50C', 'deg_50C', 'G', 'A', 'C', 'U', 'Paired',
#        'Unpaired']]

In [13]:
for a in [ 'G', 'A', 'C', 'U']:
    train[a+'_position']=[np.sum([i for i in range(len(j)) if j[i]==a])/len([i for i in range(len(j)) if j[i]==a]) for j in train['sequence']]
    test[a+'_position']=[np.sum([i for i in range(len(j)) if j[i]==a])/len([i for i in range(len(j)) if j[i]==a]) for j in test['sequence']]

In [14]:
for a in [ 'E', 'S', 'H',]:
    train[a+'_position']=[np.sum([i for i in range(len(j)) if j[i]==a])/len([i for i in range(len(j)) if j[i]==a]) for j in train['predicted_loop_type']]
    test[a+'_position']=[np.sum([i for i in range(len(j)) if j[i]==a])/len([i for i in range(len(j)) if j[i]==a]) for j in test['predicted_loop_type']]

In [15]:
for a in [ 'E', 'S', 'H',]:
    train[a+'']=[np.sum([i for i in range(len(j)) if j[i]==a])/len([i for i in range(len(j)) if j[i]==a]) for j in train['predicted_loop_type']]
    test[a+'_position']=[np.sum([i for i in range(len(j)) if j[i]==a])/len([i for i in range(len(j)) if j[i]==a]) for j in test['predicted_loop_type']]

In [16]:
a='S'
[np.sum([i for i in range(len(j)) if j[i]==a])/len([i for i in range(len(j)) if j[i]==a]) for j in train['predicted_loop_type']]

[46.32608695652174,
 42.621212121212125,
 47.395833333333336,
 44.65625,
 44.31428571428572,
 45.34,
 46.72222222222222,
 44.33870967741935,
 44.0,
 45.80357142857143,
 45.546875,
 46.84615384615385,
 51.65625,
 47.60344827586207,
 43.7,
 46.32,
 76.5,
 43.66216216216216,
 48.69230769230769,
 40.973684210526315,
 46.42307692307692,
 45.741379310344826,
 48.48148148148148,
 47.0,
 43.803030303030305,
 45.42857142857143,
 47.458333333333336,
 44.75806451612903,
 48.81818181818182,
 45.36363636363637,
 48.02272727272727,
 43.806451612903224,
 49.82142857142857,
 49.0,
 46.9,
 48.46666666666667,
 43.5,
 65.04545454545455,
 44.79032258064516,
 45.36206896551724,
 47.22222222222222,
 46.69642857142857,
 51.65625,
 47.64,
 43.66216216216216,
 44.910714285714285,
 45.520833333333336,
 46.98,
 69.77272727272727,
 44.27777777777778,
 47.07692307692308,
 48.416666666666664,
 45.410714285714285,
 43.66216216216216,
 46.9,
 76.5,
 45.74193548387097,
 44.016129032258064,
 43.72222222222222,
 45.4218

In [17]:
# import seaborn as sns
# plt.subplots(figsize=(20,10))
# sns.heatmap(train.corr()[[ 'reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C',
#        'deg_50C']],annot=True)

In [18]:
train_ex=pd.DataFrame()
for index in train.index:
    temp=pd.DataFrame()
    temp['id_seqpos']=[str(str(train['id'][index])+'_'+str(i)) for i in range(train['seq_scored'][index])]
#     temp['sequence']=[train['sequence'][index][i] for i in range(train['seq_scored'][index])]
    temp['sequence_loop']=[str(train['sequence'][index][i]+train['predicted_loop_type'][index][i]) for i in range(train['seq_scored'][index])]
    temp['structure']=[train['structure'][index][i] for i in range(train['seq_scored'][index])]
#     temp['predicted_loop_type']=[train['predicted_loop_type'][index][i] for i in range(train['seq_scored'][index])]
    for r in range(1,20):
        temp[str(str(r)+'forward_predicted_loop_type')]=[train['predicted_loop_type'][index][i+r] if i+r<train['seq_scored'][index] else -1 for i in range(train['seq_scored'][index])]
        temp[str(str(r)+'backward_predicted_loop_type')]=[train['predicted_loop_type'][index][i-r] for i in range(train['seq_scored'][index])]
        temp[str(str(r)+'forward_structure')]=[train['structure'][index][i+r] if i+r<train['seq_scored'][index] else -1 for i in range(train['seq_scored'][index])]
        temp[str(str(r)+'backward_structure')]=[train['structure'][index][i-r] for i in range(train['seq_scored'][index])]
        temp[str(str(r)+'forward_sequence')]=[train['sequence'][index][i+r] if i+r<train['seq_scored'][index] else -1 for i in range(train['seq_scored'][index])]
        temp[str(str(r)+'backward_sequence')]=[train['sequence'][index][i-r] for i in range(train['seq_scored'][index])]
    temp['E']=train['E'][index]
    temp['S']=train['S'][index]
    temp['B']=train['B'][index]
    temp['H']=train['H'][index]
    temp['I']=train['I'][index]
    temp['G']=train['G'][index]
    temp['A']=train['A'][index]
    temp['C']=train['C'][index]
    temp['U']=train['U'][index]
    temp['index']=[i for i in range(train['seq_scored'][index])]
    temp['Paired']=train['Paired'][index]
    temp['Unpaired']=train['Unpaired'][index]
    temp['G_position']=train['G_position'][index]
    temp['A_position']=train['A_position'][index]
    temp['C_position']=train['C_position'][index]
    temp['U_position']=train['U_position'][index]
    temp['E_position']=train['E_position'][index]
    temp['S_position']=train['S_position'][index]
    temp['H_position']=train['H_position'][index]
    temp['reactivity']=[train['reactivity'][index][i] for i in range(train['seq_scored'][index])]
    temp['deg_Mg_pH10']=[train['deg_Mg_pH10'][index][i] for i in range(train['seq_scored'][index])]
    temp['deg_pH10']=[train['deg_pH10'][index][i] for i in range(train['seq_scored'][index])]
    temp['deg_Mg_50C']=[train['deg_Mg_50C'][index][i] for i in range(train['seq_scored'][index])]
    temp['deg_50C']=[train['deg_50C'][index][i] for i in range(train['seq_scored'][index])]
    train_ex=train_ex.append(temp)



In [19]:
train_ex['sequence_loop'].unique()

array(['GE', 'AE', 'AS', 'GS', 'CS', 'US', 'AH', 'UH', 'CH', 'AB', 'AX',
       'AI', 'CI', 'UI', 'GH', 'GX', 'CX', 'UB', 'AM', 'UM', 'CM', 'GI',
       'GB', 'CE', 'CB', 'UE', 'UX', 'GM'], dtype=object)

In [20]:
test_ex=pd.DataFrame()
for index in test.index:
    temp=pd.DataFrame()
    temp['id_seqpos']=[str(str(test['id'][index])+'_'+str(i)) for i in range(test['seq_length'][index])]
#     temp['sequence']=[test['sequence'][index][i] for i in range(test['seq_length'][index])]
    temp['sequence_loop']=[str(test['sequence'][index][i]+test['predicted_loop_type'][index][i]) for i in range(test['seq_length'][index])]
    temp['structure']=[test['structure'][index][i] for i in range(test['seq_length'][index])]
#     temp['predicted_loop_type']=[test['predicted_loop_type'][index][i] for i in range(test['seq_length'][index])]
    
    for r in range(1,20):
        temp[str(str(r)+'forward_predicted_loop_type')]=[test['predicted_loop_type'][index][i+r] if i+r<test['seq_length'][index] else -1 for i in range(test['seq_length'][index])]
        temp[str(str(r)+'backward_predicted_loop_type')]=[test['predicted_loop_type'][index][i-r] for i in range(test['seq_length'][index])]
        temp[str(str(r)+'forward_structure')]=[test['structure'][index][i+r] if i+r<test['seq_length'][index] else -1 for i in range(test['seq_length'][index])]
        temp[str(str(r)+'backward_structure')]=[test['structure'][index][i-r] for i in range(test['seq_length'][index])]
        temp[str(str(r)+'forward_sequence')]=[test['sequence'][index][i+r] if i+r<test['seq_length'][index] else -1 for i in range(test['seq_length'][index])]
        temp[str(str(r)+'backward_sequence')]=[test['sequence'][index][i-r] for i in range(test['seq_length'][index])]
    temp['E']=test['E'][index]
    temp['S']=test['S'][index]
    temp['B']=test['B'][index]
    temp['H']=test['H'][index]
    temp['I']=test['I'][index]
    temp['G']=test['G'][index]
    temp['A']=test['A'][index]
    temp['C']=test['C'][index]
    temp['U']=test['U'][index]
    temp['index']=[i for i in range(test['seq_length'][index])]
    temp['Paired']=test['Paired'][index]
    temp['Unpaired']=test['Unpaired'][index]
    temp['G_position']=test['G_position'][index]
    temp['A_position']=test['A_position'][index]
    temp['C_position']=test['C_position'][index]
    temp['U_position']=test['U_position'][index]
    temp['E_position']=test['E_position'][index]
    temp['S_position']=test['S_position'][index]
    temp['H_position']=test['H_position'][index]
    test_ex=test_ex.append(temp)



In [21]:
result=test_ex

In [22]:
test_columns=[i for i in np.intersect1d(test_ex.columns,train_ex.columns) if i!='id_seqpos']
x_test=test_ex[test_columns]
x_t=train_ex[test_columns]
x_test=pd.get_dummies(x_test)
x_t=pd.get_dummies(x_t)
test_dum_columns=[i for i in np.intersect1d(x_t.columns,x_test.columns)]
y_t=train_ex[['reactivity', 'deg_Mg_pH10',
       'deg_pH10', 'deg_Mg_50C', 'deg_50C']]
x_t=x_t[test_dum_columns]
x_test=x_test[test_dum_columns]

In [23]:
scaler = StandardScaler()
scaler.fit(x_t)

x_t = scaler.transform(x_t)
x_test = scaler.transform(x_test)

In [24]:
x_train,x_valid,y_train,y_valid=train_test_split(x_t,y_t,test_size=0.1,shuffle=True)
print(x_train.shape)
print(y_train.shape)

(146880, 639)
(146880, 5)


In [25]:
import optuna
import xgboost as xgb
import sklearn
column='reactivity'
def objective(trial):
    column='reactivity'
    dtrain = xgb.DMatrix(x_train, label=y_train[column])
    dvalid = xgb.DMatrix(x_valid, label=y_valid[column])
    
    param = {
        "silent": 1,
#           "scale_pos_weight":trial.suggest_int("scale_pos_weight", 1, 10),
          "eval_metric": "rmse",
        "booster": "gbtree",
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
        "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
        'tree_method' : 'gpu_hist'
        
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
#         param["eta"] = trial.suggest_loguniform("eta", 1e-8, 1.0)
#         param["gamma"] = trial.suggest_loguniform("gamma", 1e-8, 1.0)
#         param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])


    # Add a callback for pruning.
#     pruning_callback = optuna.integration.XGBoostPruningCallback(trial, str("validation-"+param["eval_metric"]))
    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")])
    preds = bst.predict(dvalid)
    rmse=np.sqrt(sklearn.metrics.mean_squared_error(y_valid[column], preds))
    return rmse



In [26]:
study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2020-09-12 15:40:17,100] A new study created in memory with name: no-name-d4209755-b2c2-4801-acb0-93b8197a4e32


[15:40:20] WARNING: ../src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation-rmse:0.65657
[1]	validation-rmse:0.63423
[2]	validation-rmse:0.62204
[3]	validation-rmse:0.61518
[4]	validation-rmse:0.60850
[5]	validation-rmse:0.60496
[6]	validation-rmse:0.60147
[7]	validation-rmse:0.59959
[8]	validation-rmse:0.59690
[9]	validation-rmse:0.59554


[I 2020-09-12 15:40:22,093] Trial 0 finished with value: 0.5955348574946093 and parameters: {'lambda': 0.002148056940045521, 'alpha': 3.594312157998219e-08, 'max_depth': 4}. Best is trial 0 with value: 0.5955348574946093.


[15:40:25] WARNING: ../src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation-rmse:0.66227
[1]	validation-rmse:0.64337
[2]	validation-rmse:0.63134
[3]	validation-rmse:0.62321
[4]	validation-rmse:0.61787
[5]	validation-rmse:0.61169
[6]	validation-rmse:0.60846
[7]	validation-rmse:0.60643
[8]	validation-rmse:0.60514
[9]	validation-rmse:0.60352


[I 2020-09-12 15:40:25,976] Trial 1 finished with value: 0.6035219698641864 and parameters: {'lambda': 0.0023288476913256207, 'alpha': 1.292781155374296e-07, 'max_depth': 3}. Best is trial 0 with value: 0.5955348574946093.


...

[I 2020-09-12 15:53:42,729] Trial 197 finished with value: 0.5839914098766815 and parameters: {'lambda': 0.9765467299550719, 'alpha': 1.9590053801230678e-06, 'max_depth': 7}. Best is trial 86 with value: 0.5816870184165757.


[15:53:45] WARNING: ../src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation-rmse:0.64652
[1]	validation-rmse:0.62090
[2]	validation-rmse:0.60468
[3]	validation-rmse:0.59682
[4]	validation-rmse:0.59179
[5]	validation-rmse:0.58981
[6]	validation-rmse:0.58672
[7]	validation-rmse:0.58416
[8]	validation-rmse:0.58361
[9]	validation-rmse:0.58212


[I 2020-09-12 15:53:46,871] Trial 198 finished with value: 0.5821222455400391 and parameters: {'lambda': 0.9911194062171721, 'alpha': 2.7954441352651878e-05, 'max_depth': 7}. Best is trial 86 with value: 0.5816870184165757.


[15:53:50] WARNING: ../src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation-rmse:0.64653
[1]	validation-rmse:0.62092
[2]	validation-rmse:0.60467
[3]	validation-rmse:0.59705
[4]	validation-rmse:0.59328
[5]	validation-rmse:0.58689
[6]	validation-rmse:0.58448
[7]	validation-rmse:0.58361
[8]	validation-rmse:0.58332
[9]	validation-rmse:0.58398


[I 2020-09-12 15:53:51,345] Trial 199 finished with value: 0.5839787485015429 and parameters: {'lambda': 0.9784372431596919, 'alpha': 4.7264434280999505e-06, 'max_depth': 7}. Best is trial 86 with value: 0.5816870184165757.


In [27]:
print(study.best_params)


{'lambda': 0.9918994842168749, 'alpha': 4.636630745806852e-06, 'max_depth': 7}


In [28]:
dtrain = xgb.DMatrix(x_train, label=y_train[column])
dvalid = xgb.DMatrix(x_valid, label=y_valid[column])
dtest = xgb.DMatrix(x_test)
bst = xgb.train( study.best_params,dtrain, evals=[(dvalid, "validation")])
preds = bst.predict(dtest)
result[column]=preds

[0]	validation-rmse:0.64698
[1]	validation-rmse:0.62143
[2]	validation-rmse:0.60498
[3]	validation-rmse:0.59774
[4]	validation-rmse:0.59369
[5]	validation-rmse:0.58896
[6]	validation-rmse:0.58636
[7]	validation-rmse:0.58396
[8]	validation-rmse:0.58403
[9]	validation-rmse:0.58246


In [29]:
import optuna
import xgboost as xgb
import sklearn
column='deg_Mg_pH10'
def objective(trial):
    column='deg_Mg_pH10'
    dtrain = xgb.DMatrix(x_train, label=y_train[column])
    dvalid = xgb.DMatrix(x_valid, label=y_valid[column])

    param = {
        "silent": 1,
#           "scale_pos_weight":trial.suggest_int("scale_pos_weight", 1, 10),
          "eval_metric": "rmse",
        "booster": "gbtree",
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
        "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
        'tree_method' : 'gpu_hist'
        
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
#         param["eta"] = trial.suggest_loguniform("eta", 1e-8, 1.0)
#         param["gamma"] = trial.suggest_loguniform("gamma", 1e-8, 1.0)
#         param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])


    # Add a callback for pruning.
#     pruning_callback = optuna.integration.XGBoostPruningCallback(trial, str("validation-"+param["eval_metric"]))
    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")])
    preds = bst.predict(dvalid)
    rmse=np.sqrt(sklearn.metrics.mean_squared_error(y_valid[column], preds))
    return rmse


In [30]:
study = optuna.create_study()
study.optimize(objective, n_trials=200)


[I 2020-09-12 15:54:53,518] A new study created in memory with name: no-name-8c5455a5-290a-4159-82c0-0034267ecc83


[15:54:56] WARNING: ../src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation-rmse:0.69239
[1]	validation-rmse:0.66577
[2]	validation-rmse:0.64797
[3]	validation-rmse:0.63864
[4]	validation-rmse:0.63575
[5]	validation-rmse:0.63280
[6]	validation-rmse:0.63021
[7]	validation-rmse:0.62914
[8]	validation-rmse:0.62960
[9]	validation-rmse:0.62735


[I 2020-09-12 15:54:57,568] Trial 0 finished with value: 0.6273502691800925 and parameters: {'lambda': 9.372375267946896e-08, 'alpha': 4.230237528771322e-06, 'max_depth': 7}. Best is trial 0 with value: 0.6273502691800925.


[15:55:00] WARNING: ../src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation-rmse:0.70915
[1]	validation-rmse:0.69478
[2]	validation-rmse:0.67986
[3]	validation-rmse:0.66954
[4]	validation-rmse:0.66527
[5]	validation-rmse:0.66058
[6]	validation-rmse:0.65669
[7]	validation-rmse:0.65341
[8]	validation-rmse:0.65149
[9]	validation-rmse:0.64895


[I 2020-09-12 15:55:01,232] Trial 1 finished with value: 0.648949588211569 and parameters: {'lambda': 0.1957913887017495, 'alpha': 0.007237031160915045, 'max_depth': 3}. Best is trial 0 with value: 0.6273502691800925.


...

[I 2020-09-12 16:09:06,808] Trial 197 finished with value: 0.6182749351325789 and parameters: {'lambda': 0.3531113417410103, 'alpha': 0.00020409437230146062, 'max_depth': 8}. Best is trial 151 with value: 0.6069573953294797.


[16:09:09] WARNING: ../src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation-rmse:0.68718
[1]	validation-rmse:0.65673
[2]	validation-rmse:0.64007
[3]	validation-rmse:0.62835
[4]	validation-rmse:0.62253
[5]	validation-rmse:0.61896
[6]	validation-rmse:0.61553
[7]	validation-rmse:0.61403
[8]	validation-rmse:0.61206
[9]	validation-rmse:0.61223


[I 2020-09-12 16:09:11,032] Trial 198 finished with value: 0.6122267916901619 and parameters: {'lambda': 0.9624985122236571, 'alpha': 0.21795789886308842, 'max_depth': 8}. Best is trial 151 with value: 0.6069573953294797.


[16:09:14] WARNING: ../src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation-rmse:0.68582
[1]	validation-rmse:0.65854
[2]	validation-rmse:0.64114
[3]	validation-rmse:0.63198
[4]	validation-rmse:0.62864
[5]	validation-rmse:0.62664
[6]	validation-rmse:0.62512
[7]	validation-rmse:0.62389
[8]	validation-rmse:0.62217
[9]	validation-rmse:0.62075


[I 2020-09-12 16:09:15,724] Trial 199 finished with value: 0.6207546541711425 and parameters: {'lambda': 0.6351828496337854, 'alpha': 0.0001149796962363024, 'max_depth': 9}. Best is trial 151 with value: 0.6069573953294797.


In [31]:
study.best_params

{'lambda': 0.9633838060118881, 'alpha': 0.00021902382493614787, 'max_depth': 8}

In [32]:
dtrain = xgb.DMatrix(x_train, label=y_train[column])
dvalid = xgb.DMatrix(x_valid, label=y_valid[column])
dtest = xgb.DMatrix(x_test)
bst = xgb.train( study.best_params,dtrain, evals=[(dvalid, "validation")])
preds = bst.predict(dtest)
result[column]=preds

[0]	validation-rmse:0.68680
[1]	validation-rmse:0.65614
[2]	validation-rmse:0.63878
[3]	validation-rmse:0.62724
[4]	validation-rmse:0.62274
[5]	validation-rmse:0.61954
[6]	validation-rmse:0.61717
[7]	validation-rmse:0.61624
[8]	validation-rmse:0.61408
[9]	validation-rmse:0.61266


In [33]:
import optuna
import xgboost as xgb
import sklearn
column='deg_Mg_50C'
def objective(trial):
    column='deg_Mg_50C'
    dtrain = xgb.DMatrix(x_train, label=y_train[column])
    dvalid = xgb.DMatrix(x_valid, label=y_valid[column])

    param = {
        "silent": 1,
#           "scale_pos_weight":trial.suggest_int("scale_pos_weight", 1, 10),
          "eval_metric": "rmse",
        "booster": "gbtree",
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
        "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
        'tree_method' : 'gpu_hist'
        
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
#         param["eta"] = trial.suggest_loguniform("eta", 1e-8, 1.0)
#         param["gamma"] = trial.suggest_loguniform("gamma", 1e-8, 1.0)
#         param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])


    # Add a callback for pruning.
#     pruning_callback = optuna.integration.XGBoostPruningCallback(trial, str("validation-"+param["eval_metric"]))
    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")])
    preds = bst.predict(dvalid)
    rmse=np.sqrt(sklearn.metrics.mean_squared_error(y_valid[column], preds))
    return rmse


In [34]:
study = optuna.create_study()
study.optimize(objective, n_trials=200)


[I 2020-09-12 16:10:32,295] A new study created in memory with name: no-name-603d9f11-764f-46ca-87f0-2b722b48ddf5


[16:10:35] WARNING: ../src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation-rmse:0.80136
[1]	validation-rmse:0.77819
[2]	validation-rmse:0.76297
[3]	validation-rmse:0.76555
[4]	validation-rmse:0.76653
[5]	validation-rmse:0.77806
[6]	validation-rmse:0.77815
[7]	validation-rmse:0.77917
[8]	validation-rmse:0.77910
[9]	validation-rmse:0.78957


[I 2020-09-12 16:10:36,278] Trial 0 finished with value: 0.7895663237380048 and parameters: {'lambda': 2.48125466972805e-08, 'alpha': 0.19153856724611232, 'max_depth': 7}. Best is trial 0 with value: 0.7895663237380048.


[16:10:39] WARNING: ../src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation-rmse:0.84099
[1]	validation-rmse:0.83273
[2]	validation-rmse:0.82767
[3]	validation-rmse:0.82429
[4]	validation-rmse:0.82115
[5]	validation-rmse:0.81880
[6]	validation-rmse:0.81659
[7]	validation-rmse:0.81411
[8]	validation-rmse:0.81267
[9]	validation-rmse:0.81087


[I 2020-09-12 16:10:39,996] Trial 1 finished with value: 0.8108705794977218 and parameters: {'lambda': 0.07577524823854367, 'alpha': 0.011065235720437737, 'max_depth': 1}. Best is trial 0 with value: 0.7895663237380048.


...

[I 2020-09-12 16:25:19,019] Trial 197 finished with value: 0.7720582023709474 and parameters: {'lambda': 0.5056482741625647, 'alpha': 2.4910835389865916e-05, 'max_depth': 9}. Best is trial 32 with value: 0.7385277348675058.


[16:25:22] WARNING: ../src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation-rmse:0.79595
[1]	validation-rmse:0.76816
[2]	validation-rmse:0.75730
[3]	validation-rmse:0.75328
[4]	validation-rmse:0.75310
[5]	validation-rmse:0.75600
[6]	validation-rmse:0.75557
[7]	validation-rmse:0.75277
[8]	validation-rmse:0.75410
[9]	validation-rmse:0.74892


[I 2020-09-12 16:25:23,677] Trial 198 finished with value: 0.7489231830279761 and parameters: {'lambda': 0.6948618940809428, 'alpha': 4.651439038329668e-05, 'max_depth': 9}. Best is trial 32 with value: 0.7385277348675058.


[16:25:26] WARNING: ../src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation-rmse:0.79588
[1]	validation-rmse:0.76698
[2]	validation-rmse:0.75476
[3]	validation-rmse:0.74768
[4]	validation-rmse:0.74300
[5]	validation-rmse:0.73957
[6]	validation-rmse:0.74265
[7]	validation-rmse:0.74004
[8]	validation-rmse:0.73819
[9]	validation-rmse:0.74396


[I 2020-09-12 16:25:28,238] Trial 199 finished with value: 0.7439585082489348 and parameters: {'lambda': 0.9960370924483456, 'alpha': 0.00017001483242511394, 'max_depth': 9}. Best is trial 32 with value: 0.7385277348675058.


In [35]:
study.best_params

{'lambda': 0.7788572927251278, 'alpha': 2.8954093805668896e-08, 'max_depth': 9}

In [36]:
dtrain = xgb.DMatrix(x_train, label=y_train[column])
dvalid = xgb.DMatrix(x_valid, label=y_valid[column])
dtest = xgb.DMatrix(x_test)
bst = xgb.train( study.best_params,dtrain, evals=[(dvalid, "validation")])
preds = bst.predict(dtest)
result[column]=preds

[0]	validation-rmse:0.80589
[1]	validation-rmse:0.77696
[2]	validation-rmse:0.76475
[3]	validation-rmse:0.76316
[4]	validation-rmse:0.76067
[5]	validation-rmse:0.75916
[6]	validation-rmse:0.76226
[7]	validation-rmse:0.76416
[8]	validation-rmse:0.77049
[9]	validation-rmse:0.77497


In [37]:
result.head()

,id_seqpos,sequence_loop,structure,1forward_predicted_loop_type,1backward_predicted_loop_type,1forward_structure,1backward_structure,1forward_sequence,1backward_sequence,2forward_predicted_loop_type,...,G_position,A_position,C_position,U_position,E_position,S_position,H_position,reactivity,deg_Mg_pH10,deg_Mg_50C
0,id_00073f8be_0,GE,.,E,E,.,.,G,C,E,...,43.0,56.489362,59.35,49.055556,75.222222,44.283333,46.5,0.624838,0.619721,2.215972
1,id_00073f8be_1,GE,.,E,E,.,.,A,G,E,...,43.0,56.489362,59.35,49.055556,75.222222,44.283333,46.5,1.727007,2.888908,2.907256
2,id_00073f8be_2,AE,.,E,E,.,.,A,G,E,...,43.0,56.489362,59.35,49.055556,75.222222,44.283333,46.5,1.208527,0.690989,1.306046
3,id_00073f8be_3,AE,.,E,E,.,.,A,A,E,...,43.0,56.489362,59.35,49.055556,75.222222,44.283333,46.5,1.239894,0.994922,1.454854
4,id_00073f8be_4,AE,.,E,E,.,.,A,A,S,...,43.0,56.489362,59.35,49.055556,75.222222,44.283333,46.5,0.994373,0.645619,0.823127


In [38]:
result['deg_pH10']=0
result['deg_50C']=0
result[['id_seqpos','reactivity', 'deg_Mg_pH10',
       'deg_pH10', 'deg_Mg_50C', 'deg_50C']].head()

,id_seqpos,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,id_00073f8be_0,0.624838,0.619721,0,2.215972,0
1,id_00073f8be_1,1.727007,2.888908,0,2.907256,0
2,id_00073f8be_2,1.208527,0.690989,0,1.306046,0
3,id_00073f8be_3,1.239894,0.994922,0,1.454854,0
4,id_00073f8be_4,0.994373,0.645619,0,0.823127,0


In [39]:
result[['id_seqpos','reactivity', 'deg_Mg_pH10',
       'deg_pH10', 'deg_Mg_50C', 'deg_50C']].to_csv('submission.csv',index=False)